In [1]:
import re
import os
import requests
import zipfile
import math
import pandas as pd
import numpy as np
from tqdm import tqdm

In [24]:
root_url = 'http://www.kdnuggets.com/data_mining_course/data/'
root_data = 'data'
train_data_file = 'ALL_AML_gr.thr.train.csv'
data_file = 'genes-leukemia.csv'
reduced_data_file = 'genes-reduced.csv'
randomized_data_file = 'genes-reduced-randomized'

In [3]:
def unzip_file(file_addr,output_dir,remove=False) :
    with zipfile.ZipFile(file_addr,"r") as zip_ref:
        zip_ref.extractall(output_dir)
    if remove :  
        os.remove(file_addr)
    return os.listdir(output_dir)

def download_file(url,output_file,unzip=False,output_dir='') :
    if not os.path.exists(os.path.join(output_dir,output_file)) :
        # Streaming, so we can iterate over the response.
        r = requests.get(url, stream=True)

        # Total size in bytes.
        total_size = int(r.headers.get('content-length', 0)); 
        block_size = 1024
        wrote = 0 
        with open(os.path.join(output_dir,output_file), 'wb') as f:
            for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='KB', unit_scale=True):
                wrote = wrote  + len(data)
                f.write(data)
        if total_size != 0 and wrote != total_size:
            print("ERROR, something went wrong")
    if unzip :
        return unzip_file(os.path.join(output_dir,output_file),output_dir,True)
    return os.path.join(output_dir,output_file)

file = download_file(root_url + data_file,data_file,False,root_data)

In [4]:
data = pd.read_csv(file)
data.head(5)

,SNUM,CLASS,BM_PB,TB_if_ALL,FAB_if_AML,Year,Gender,pct_Blasts,Treatment_Response,PS,...,AF012024_s,D26156_s,M84371_rna1_s,M31211_s,U09087_s,U26266_s,L49229_f,M31523,M28170,U29175
0,s1,ALL,BM,B-cell,?,1996,M,?,?,1.00,...,257,1595,2911,601,358,289,337,1320,397,1582
1,s2,ALL,BM,T-cell,?,?,M,?,?,0.41,...,46,822,575,435,82,288,131,898,20,624
2,s3,ALL,BM,T-cell,?,?,M,?,?,0.87,...,139,1452,905,547,263,447,529,597,183,753
3,s4,ALL,BM,B-cell,?,?,?,?,?,0.91,...,168,654,2038,472,218,424,422,1644,363,743
4,s5,ALL,BM,B-cell,?,?,?,?,?,0.89,...,94,1011,1871,661,186,364,354,1322,251,626


In [5]:
none_missing_tr_data = data[data['Treatment_Response'] != '?']
none_missing_tr_data

,SNUM,CLASS,BM_PB,TB_if_ALL,FAB_if_AML,Year,Gender,pct_Blasts,Treatment_Response,PS,...,AF012024_s,D26156_s,M84371_rna1_s,M31211_s,U09087_s,U26266_s,L49229_f,M31523,M28170,U29175
27,s28,AML,BM,?,M2,?,?,79,Failure,0.44,...,43,893,443,300,91,151,20,353,20,429
28,s29,AML,BM,?,M2,?,?,34,Failure,0.74,...,50,624,225,237,58,66,20,279,35,314
29,s30,AML,BM,?,M5,?,?,93,Failure,0.80,...,20,388,251,100,20,20,60,250,20,208
30,s31,AML,BM,?,M4,?,?,77,Failure,0.61,...,45,432,595,192,61,20,56,381,20,385
31,s32,AML,BM,?,M1,?,?,86,Failure,0.47,...,68,706,863,339,50,192,88,671,20,688
32,s33,AML,BM,?,M2,?,?,70,Failure,0.89,...,20,736,678,59,20,20,20,200,20,248
33,s34,AML,BM,?,M2,?,?,77,Success,0.64,...,28,302,469,153,34,282,20,299,20,454
34,s35,AML,BM,?,M1,?,?,67,Success,0.21,...,50,201,548,20,48,43,38,389,67,391
35,s36,AML,BM,?,M5,?,?,76,Success,0.94,...,29,667,763,88,74,64,20,126,20,412
36,s37,AML,BM,?,M2,?,?,44,Success,0.95,...,98,697,466,139,52,20,93,190,20,393


In [6]:
print('There is {} samples with none missing treatment_response value in the data' 
      .format(none_missing_tr_data.shape[0]))

There is 15 samples with none missing treatment_response value in the data


In [7]:
none_missing_tr_data.describe()

,PS,D49950,D63880,J03473,J05243,L13278,L47738,M21551_rna1,M55150,M62762,...,AF012024_s,D26156_s,M84371_rna1_s,M31211_s,U09087_s,U26266_s,L49229_f,M31523,M28170,U29175
count,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,0.72600,230.333333,200.266667,542.066667,151.466667,24.933333,255.800000,429.800000,1717.866667,3069.733333,...,44.600000,592.000000,542.533333,149.466667,45.066667,95.533333,41.266667,283.133333,31.733333,359.800000
std,0.22959,97.389987,100.603512,204.579871,158.522133,11.208840,214.475839,114.143519,423.082036,976.326897,...,24.094457,270.763207,198.136269,95.727938,21.137194,110.494904,24.760183,143.173654,25.417842,122.862409
min,0.21000,70.000000,20.000000,109.000000,20.000000,20.000000,20.000000,259.000000,801.000000,1125.000000,...,20.000000,20.000000,225.000000,20.000000,20.000000,20.000000,20.000000,110.000000,20.000000,190.000000
25%,0.61000,149.500000,133.500000,428.000000,52.500000,20.000000,91.500000,340.500000,1524.000000,2411.000000,...,28.500000,410.000000,454.500000,72.500000,26.500000,20.000000,20.000000,193.500000,20.000000,268.000000
50%,0.74000,259.000000,185.000000,500.000000,102.000000,20.000000,201.000000,415.000000,1707.000000,3266.000000,...,41.000000,667.000000,530.000000,139.000000,48.000000,43.000000,38.000000,275.000000,20.000000,382.000000
75%,0.91500,304.000000,278.000000,673.000000,221.500000,21.000000,447.500000,486.000000,1912.000000,3573.500000,...,50.000000,728.500000,677.000000,214.500000,55.000000,135.000000,54.000000,367.000000,27.500000,402.500000
max,1.00000,355.000000,410.000000,871.000000,628.000000,60.000000,683.000000,672.000000,2693.000000,4647.000000,...,98.000000,1024.000000,863.000000,339.000000,91.000000,376.000000,93.000000,671.000000,110.000000,688.000000


### Why is it not correct to build predictive models for TREATMENT_RESPONSE using records where it is missing?
Because we need labels for any supervised learning algorithm, but when treatment_response is missing, we don't have any label.


In [8]:
one_value_columns = []
for name in none_missing_tr_data.columns :
    if len(none_missing_tr_data.groupby(name).size()) == 1 :
        one_value_columns.append(name)
reduced_data = none_missing_tr_data.drop(one_value_columns,axis=1)
reduced_data.to_csv(os.path.join(root_data,reduced_data_file),index=False)

In [9]:
print('Samples Field that i should keep : \n{}'.format([column for column in none_missing_tr_data.columns if column not in one_value_columns]))

Samples Field that i should keep : 
['SNUM', 'BM_PB', 'FAB_if_AML', 'pct_Blasts', 'Treatment_Response', 'PS', 'D49950', 'D63880', 'J03473', 'J05243', 'L13278', 'L47738', 'M21551_rna1', 'M55150', 'M62762', 'M81933', 'M91432', 'S50223', 'U12471_cds1', 'U32944', 'U35451', 'U50136_rna1', 'U53468', 'U72342', 'U82759', 'X15949', 'X52142', 'X56411_rna1', 'X63469', 'X74262', 'X76061', 'X76648', 'X95735', 'Y08612', 'Y12670', 'D38073', 'AF012024_s', 'D26156_s', 'M84371_rna1_s', 'M31211_s', 'U09087_s', 'U26266_s', 'L49229_f', 'M31523', 'M28170', 'U29175']


 Build a CART Model using leave-one-out cross validation.

what tree do you get?

![title](images/tree_architecture.PNG)

what is the expected error rate?

![title](images/tree_error_rate.PNG)

what are the important variables and their relative importance, according to CART?

There is no relative variables according to j4.8, because it just used one of the attribute for predection.

Remove the top predictor -- and re-run the CART -- what do you get?

tree :
![title](images/tree_architecture_after_remove.PNG)

metrics :
![title](images/tree_error_rate_after_remove.PNG)

Randomize the TREATMENT_RESPONSE variable 10 times and re-run the CART for each randomized class.

In [26]:
reduced_data_randomized = reduced_data[:]
for i in range(10) :
    tmp_arr = np.array(reduced_data_randomized['Treatment_Response'])
    np.random.shuffle(tmp_arr)
    reduced_data_randomized['Treatment_Response'] = tmp_arr
    reduced_data.to_csv(os.path.join(root_data,randomized_data_file + str(i) + '.csv'),index=False)

First Result :

![title](images/randomized_metrics_0.PNG)

![title](images/randomized_tree_0.PNG)

Based on the results in Q8, do you think the tree that you found with the original data is significant?